## Random forest

In [202]:
import numpy as np
from sklearn.metrics import make_scorer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import log_loss
import xgboost as xgb
import category_encoders
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
import scipy.stats as stats
import sklearn.metrics

In [203]:
sklearn.metrics.SCORERS.keys()

dict_keys(['explained_variance', 'r2', 'max_error', 'neg_median_absolute_error', 'neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_root_mean_squared_error', 'neg_mean_poisson_deviance', 'neg_mean_gamma_deviance', 'accuracy', 'roc_auc', 'roc_auc_ovr', 'roc_auc_ovo', 'roc_auc_ovr_weighted', 'roc_auc_ovo_weighted', 'balanced_accuracy', 'average_precision', 'neg_log_loss', 'neg_brier_score', 'adjusted_rand_score', 'homogeneity_score', 'completeness_score', 'v_measure_score', 'mutual_info_score', 'adjusted_mutual_info_score', 'normalized_mutual_info_score', 'fowlkes_mallows_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'jaccard', 'jaccard_macro', 'jaccard_micro', 'jaccard_samples', 'jaccard_weighted'])

In [204]:
df_train = pd.read_csv('data/train.csv')
df_test = pd.read_csv('data/test.csv')
features = pd.read_csv('data/features.csv')

In [205]:
target = features["Stage"]
features.drop(columns=["Stage"], inplace=True)

In [206]:
parameters = {'n_estimators':[50, 100, 150, 200, 250, 300]}
#parameters = {'n_estimators':[10, 20, 30, 40, 50, 60, 70, 80, 90, 100]}
#parameters={'n_estimators':[1,2,3,4,5,6,7,8,9,10]}

grid = GridSearchCV(RandomForestClassifier(),parameters,cv=2,verbose=0, scoring='neg_log_loss')


In [207]:
xtr,xte,ytr,yte=train_test_split(features,target)

In [208]:
grid.fit(xtr,ytr)

GridSearchCV(cv=2, estimator=RandomForestClassifier(),
             param_grid={'n_estimators': [50, 100, 150, 200, 250, 300]},
             scoring='neg_log_loss')

In [209]:
print(grid.best_score_, grid.best_estimator_)

-0.2753940780830135 RandomForestClassifier(n_estimators=300)


In [210]:
grid.score(X=xte,y=yte)

-0.2519719219472776

In [211]:
rfc = grid.best_estimator_
importance = rfc.feature_importances_
pd.DataFrame([features.columns,importance])

,0,1,2,3,4,5,6,7,8,9,...,68,69,70,71,72,73,74,75,76,77
0,Unnamed: 0,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,ASP,ASP_(converted),TRF,Total_Amount,...,Opportunity_Owner_4_target_binary_enc,Opportunity_Owner_5_target_binary_enc,Opportunity_Owner_6_target_binary_enc,Account_Owner_0_target_binary_enc,Account_Owner_1_target_binary_enc,Account_Owner_2_target_binary_enc,Account_Owner_3_target_binary_enc,Account_Owner_4_target_binary_enc,Account_Owner_5_target_binary_enc,Account_Owner_6_target_binary_enc
1,0.0577103,0.00608136,0.0306224,0.00856097,0.00631228,0,0.0312103,0.0306904,0.0440624,0.0604387,...,0.00443739,0.00513761,0.00477529,0,0.00179412,0.00366224,0.00346612,0.00380268,0.00378989,0.00501361


In [212]:
best_features = features.loc[:,["ASP_(converted)","ASP","TRF","Pricing, Delivery_Terms_Approved","Total_Amount",\
                                 "Total_Taxable_Amount"]]
                
best_features.head()


,ASP_(converted),ASP,TRF,"Pricing, Delivery_Terms_Approved",Total_Amount,Total_Taxable_Amount
0,0.35629,0.315,2,0,662287.5,662287.5
1,0.49000,0.490,20,1,9803430.0,9803430.0
2,0.68000,0.680,0,0,170931.6,341863.2
3,0.68000,0.680,0,0,170931.6,341863.2
4,0.74000,0.740,3,0,1886852.0,31410780.0


In [213]:
xtrb,xteb,ytrb,yteb=train_test_split(best_features,target)
grid.fit(xtrb,ytrb)
print(grid.best_score_, grid.best_estimator_)

-0.42294974823509 RandomForestClassifier(n_estimators=300)


In [214]:
grid.score(X=xteb,y=yteb)

-0.40510547135810865

In [215]:
rfc = grid.best_estimator_
importance = rfc.feature_importances_
pd.DataFrame([best_features.columns,importance])

,0,1,2,3,4,5
0,ASP_(converted),ASP,TRF,"Pricing, Delivery_Terms_Approved",Total_Amount,Total_Taxable_Amount
1,0.141259,0.162679,0.109701,0.0773636,0.266395,0.242602
